In [1]:
import itertools
import pandas as pd
import numpy as np

In [ ]:
def get_all_substrings(input_string):
  length = len(input_string)
  return set([input_string[i:j+1] for i in range(length) for j in range(i,length)])

In [ ]:
def split_string(a_string):
    return ["x = x.split(\"{}\")".format(sep) for sep in get_all_substrings(a_string) if len(sep) <= 2]

In [ ]:
split_string("abc d")

In [ ]:
def get_select_combinations(a_list):
    list_of_list = [list(itertools.combinations(range(len(a_list)), k)) for k in range(1, len(a_list) + 1)]
    reverse_list_of_list = [list(itertools.combinations(np.linspace(-len(a_list), -1, len(a_list)).astype(int), k)) for k in range(1, len(a_list) + 1)]
    select_list = list(itertools.chain.from_iterable(list_of_list + reverse_list_of_list))
    func_str_list = ["x = [x[i] for i in {}]".format(selected) for selected in select_list if len(selected) <= 3]
    return func_str_list

In [ ]:
get_select_combinations(["a", "c", "b"])

In [ ]:
def get_permutations(a_list):
    permutations_list = list(itertools.permutations(range(len(a_list))))
    func_str_list = ["x = [x[i] for i in {}]".format(permutation) for permutation in permutations_list]
    return func_str_list

In [ ]:
get_permutations(["a", "c", "b"])

In [ ]:
def join_list_to_string(a_list, output_string):
    return ["x = \"{}\".join(x)".format(sep) for sep in list(get_all_substrings(output_string)) + [""] if len(sep) <= 2]

In [ ]:
join_list_to_string(["a", "c", "b"], "abds")

In [ ]:
def apply_meta_func(input_obj, func):
    func_list = func(input_obj)
    def apply_func(func):
        d = {"x": input_obj}
        exec(func, d)
        return d["x"]
    obj_list = list(map(apply_func, func_list))
    return func_list, obj_list 

In [ ]:
def apply_meta_func_on_group(group, meta_func):
    input_obj = group.iloc[0]["obj"]
    func_str = group.iloc[0]["func_str"]
    func_list, obj_list = apply_meta_func(input_obj, meta_func)
    output_data_frame = pd.DataFrame([obj_list, func_list]).T
    output_data_frame.columns = ["obj", "func_str"]
    func_str += "\n" + output_data_frame["func_str"]
    output_data_frame["func_str"] = func_str    
    return output_data_frame

In [ ]:
def apply_func_on_input_output_example(row, input_string, output_string):
    func_str = row
    d = {"x": input_string}
    try:
        exec(func_str, d)
        return d["x"] == output_string
    except:
        return False

In [ ]:
def code(input_output_examples_list, is_log=False):
    input_string = input_output_examples_list[0][0]
    output_string = input_output_examples_list[0][1]
    input_obj_list = [input_string]
    input_func_str_list = [""]
    func_data_frame = pd.DataFrame([input_obj_list, input_func_str_list]).T
    func_data_frame.columns = ["obj", "func_str"]
    
    join_list_to_string_with_output = lambda a_list: join_list_to_string(a_list, output_string)
    
    if is_log:
        meta_func_list = [split_string, get_select_combinations, join_list_to_string_with_output]
    else:
        meta_func_list = [split_string, get_select_combinations, get_permutations, join_list_to_string_with_output]    
    
    for meta_func in meta_func_list:
        func_data_frame = func_data_frame.groupby(["func_str"]).apply(apply_meta_func_on_group, meta_func)
        func_data_frame.reset_index(drop=True, inplace=True)
        
    print("total number of functions: {}".format(len(func_data_frame)))
        
    possible_code_series = func_data_frame[func_data_frame.obj == output_string]["func_str"]
    
    for input_output_example in input_output_examples_list[1:]:
        is_good_func_series = possible_code_series.apply(apply_func_on_input_output_example, args=(input_output_example))
        possible_code_series = possible_code_series[is_good_func_series]
        
    return possible_code_series

In [ ]:
input_output_examples_list = [("aa bb cc", "bbmmccmmaa"), ("ll dd ff", "ddmmffmmll")]
possible_code_series = code(input_output_examples_list)
possible_code_series.apply(print)

In [ ]:
input_output_examples_list = [("ffllnn", "nnff"), ("kkllmm", "mmkk")]
possible_code_series = code(input_output_examples_list)
possible_code_series.apply(print)

# logs

In [ ]:

# file_path = r".\log_example.txt"

In [ ]:
def get_max_length_common_string(string_1, string_2):
    union_list = [substring for substring in get_all_substrings(string_1) if substring in string_2]
    common_string = union_list[pd.Series(union_list).apply(len).argmax()]
    return common_string

In [ ]:
def get_info_from_log(log_lines, output_strings_list):
    log_lines = [line.replace("\n", "") for line in log_lines]
    interesting_lines = []
    for line in log_lines:
        for output_string in output_strings_list:
            if output_string in line:        
                interesting_lines.append((line, output_string))
                
    common_string = get_max_length_common_string(interesting_lines[0][0], interesting_lines[1][0])
    
    common_string_no_numeric_trail = common_string[:(-pd.Series(list(common_string)).str.isnumeric().values[::-1].argmin())]
    
    input_output_examples_list = interesting_lines
    possible_code_series = code(input_output_examples_list, True)
    
    possible_code_series.apply(print)
    
    inner_func = "\n".join(["\t" + line for line in possible_code_series.iloc[0].split("\n") if line != ""] )
    
    output_list = []
    for line in log_lines:
        if common_string_no_numeric_trail in line:
            d = {"x": line}
            exec(possible_code_series.iloc[0], d)            
            output_list.append(d["x"])
            
    func = """
output_list = []
for line in log_lines:
    if \"{}\" in line:
        x = line
{}
        output_list.append(x)
""".format(common_string_no_numeric_trail, inner_func)
    
    return output_list, func

In [ ]:
file_path = r".\log_example.txt"
log_lines = open(file_path, "r").readlines()
log_lines

In [ ]:
output_strings_list = ["port 40009", "port 48849"]

output_list, func = get_info_from_log(log_lines, output_strings_list)

display(output_list)
print(func)